In [3]:
# 构造一个sparseTensor
import tensorflow as tf
sparse_tensor=tf.sparse.SparseTensor(indices=[[0, 0], [1, 2]], values=[1, 2], dense_shape=[3, 4])
dense_tensor=tf.constant([[1, 0, 0, 0],
 [0, 0, 2, 0],
 [0, 0, 0, 0]])

print(sparse_tensor)
print(dense_tensor)

SparseTensor(indices=tf.Tensor(
[[0 0]
 [1 2]], shape=(2, 2), dtype=int64), values=tf.Tensor([1 2], shape=(2,), dtype=int32), dense_shape=tf.Tensor([3 4], shape=(2,), dtype=int64))
tf.Tensor(
[[1 0 0 0]
 [0 0 2 0]
 [0 0 0 0]], shape=(3, 4), dtype=int32)


In [5]:
import tensorflow as tf
# sparse和tensor互转

dense_tensor=tf.constant([[1, 0, 0, 0],
 [0, 0, 2, 0],
 [0, 0, 0, 0]])
arr_idx = tf.where(tf.not_equal(dense_tensor, 0))
# gather_nd这个操作就是根据arr_idx的指示，从dense_tensor提取值，最终的结果shape由arr_idx决定
# 而从arr_idx的0轴来看，每个元素都是一个坐标，因此arr_values就是一个一维度向量
arr_values=tf.gather_nd(dense_tensor, arr_idx)
sparse_tensor = tf.SparseTensor(arr_idx,arr_values,dense_shape=dense_tensor.shape)

print(arr_idx)
print(arr_values)
print(sparse_tensor)
print(tf.sparse.to_dense(sparse_tensor))

tf.Tensor(
[[0 0]
 [1 2]], shape=(2, 2), dtype=int64)
tf.Tensor([1 2], shape=(2,), dtype=int32)
SparseTensor(indices=tf.Tensor(
[[0 0]
 [1 2]], shape=(2, 2), dtype=int64), values=tf.Tensor([1 2], shape=(2,), dtype=int32), dense_shape=tf.Tensor([3 4], shape=(2,), dtype=int64))
tf.Tensor(
[[1 0 0 0]
 [0 0 2 0]
 [0 0 0 0]], shape=(3, 4), dtype=int32)


In [16]:
# sparseTensor与dataset
# dataset也可以持有sparseTensor哦

import tensorflow as tf
import numpy as np
ds=tf.data.Dataset.from_tensor_slices([np.reshape(np.random.choice([0,1],size=100),newshape=(10,10))])

def dense_to_sparse(dense_tensor):
    idx = tf.where(tf.not_equal(dense_tensor, 0))
    values=tf.gather_nd(dense_tensor, idx)
    return tf.SparseTensor(idx,values,dense_shape=dense_tensor.shape)

# for sample in ds:
#     print(sample)

# 但是需要注意，此时sparse_ds里只有一个item了，就是一个完整的sparse_tensor
sparse_ds=ds.map(dense_to_sparse)
for sample in sparse_ds:
    print("only one data")
    print(sample)

only one data
SparseTensor(indices=tf.Tensor(
[[0 2]
 [0 3]
 [0 4]
 [0 5]
 [0 6]
 [0 7]
 [0 8]
 [1 0]
 [1 3]
 [1 5]
 [1 7]
 [2 1]
 [2 3]
 [2 6]
 [2 7]
 [2 8]
 [2 9]
 [3 0]
 [3 1]
 [3 2]
 [3 4]
 [3 5]
 [3 7]
 [4 3]
 [4 8]
 [5 0]
 [5 1]
 [5 2]
 [5 3]
 [5 6]
 [5 7]
 [5 8]
 [6 1]
 [6 2]
 [6 3]
 [6 4]
 [6 6]
 [6 8]
 [7 0]
 [7 1]
 [7 2]
 [7 4]
 [7 5]
 [7 6]
 [7 8]
 [7 9]
 [8 0]
 [8 1]
 [8 2]
 [8 3]
 [8 6]
 [9 0]
 [9 2]
 [9 8]
 [9 9]], shape=(55, 2), dtype=int64), values=tf.Tensor(
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1], shape=(55,), dtype=int32), dense_shape=tf.Tensor([10 10], shape=(2,), dtype=int64))


In [36]:
# 在tf1里，是不能将一个shape为(None,10)的tensor转化为sparse Tensor
# 因为形状必须是明确的
# 但是在tf2里，由于动态图的存在，多数tensor的shape都是确定好的，所以不太容易存在tf1中的问题

ds=tf.data.Dataset.from_tensor_slices(np.random.choice([0,1],size=[10,10])).batch(5)

def dense_to_sparse(dense_tensor):
    idx = tf.where(tf.not_equal(dense_tensor, 0))
    values=tf.gather_nd(dense_tensor, idx)
    return tf.SparseTensor(idx,values,dense_shape=dense_tensor.shape)

for batch in ds:
    # batch的形状是确定的
    print(dense_to_sparse(batch))

SparseTensor(indices=tf.Tensor(
[[0 0]
 [0 2]
 [0 3]
 [0 4]
 [0 6]
 [0 7]
 [1 1]
 [1 3]
 [1 4]
 [1 5]
 [2 0]
 [2 5]
 [2 6]
 [2 7]
 [2 8]
 [3 2]
 [3 3]
 [3 4]
 [3 5]
 [3 6]
 [3 7]
 [3 8]
 [4 1]
 [4 3]], shape=(24, 2), dtype=int64), values=tf.Tensor([1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1], shape=(24,), dtype=int64), dense_shape=tf.Tensor([ 5 10], shape=(2,), dtype=int64))
SparseTensor(indices=tf.Tensor(
[[0 0]
 [0 1]
 [0 2]
 [0 3]
 [0 6]
 [0 7]
 [1 1]
 [1 3]
 [1 5]
 [1 7]
 [2 0]
 [2 2]
 [2 3]
 [2 4]
 [2 7]
 [2 8]
 [2 9]
 [3 6]
 [3 9]
 [4 1]
 [4 4]
 [4 7]], shape=(22, 2), dtype=int64), values=tf.Tensor([1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1], shape=(22,), dtype=int64), dense_shape=tf.Tensor([ 5 10], shape=(2,), dtype=int64))


In [34]:
ds

<BatchDataset shapes: (None, 10), types: tf.int64>